In [ ]:
!pip install pandas

In [ ]:
!pip install psycopg2 #install postgres driver

In [5]:
DB_HOST = 'localhost'
DB_PORT = 5432
DB_USERNAME = 'group11'
DB_PASSWORD = 'xxxxxx'
DB_NAME = 'tweets'

In [6]:
region1 = {
  "germany1": {
    "geocode": "52.739184,10.627171,236.4km",
    "language": "de"
  },
  "germany2": {
    "geocode": "51.753988,12.457496,142.434km",
    "language": "de"
  },
  "germany3": {
    "geocode": "51.358081,7.207764,69.343km",
    "language": "de"
  },
  "germany4": {
    "geocode": "50.496732,8.736488,170.602km",
    "language": "de"
  },
  "germany5": {
    "geocode": "49.000985,10.452385,165.858km",
    "language": "de"
  },
  "germany6": {
    "geocode": "48.054800,8.147429,39.891km",
    "language": "de"
  }
}

region2 = {
  "netherlands1": {
    "geocode": "52.944864,4.316700,163km",
    "language": "nl"
  },
  "netherlands2": {
    "geocode": "52.314066,6.025588,49.7km",
    "language": "nl"
  },
  "netherlands3": {
    "geocode": "51.532413,3.796285,28.8km",
    "language": "nl"
  },
  "netherlands4": {
    "geocode": "51.557595,5.585039,32km",
    "language": "nl"
  },
  "netherlands5": {
    "geocode": "50.888106,5.843230,15km",
    "language": "nl"
  }
}

In [7]:
def region_sql_query(region):
    """Get all NL tweets with Dutch language"""
    return """
SELECT
tweet_id,
inserted,

location_query,
location_id,

tweet_body->'created_at' as created_at,
tweet_body->'user'->'id_str' as user_id_str,
tweet_body->'user'->'screen_name' as user_screen_name,
tweet_body->'user'->'followers_count' as followers_count,

tweet_body->'entities'->'user_mentions'->0->>'screen_name' as user_mention_screen_name,
tweet_body->'entities'->'user_mentions'->0->>'id' as user_mention_id_str,

tweet_body->'in_reply_to_status_id' as in_reply_to_status_id,
tweet_body->'in_reply_to_status_id_str' as in_reply_to_status_id_str,
tweet_body->'in_reply_to_user_id_str' as in_reply_to_user_id_str,
tweet_body->'in_reply_to_screen_name' as in_reply_to_screen_name,

tweet_body->'retweeted_status'->'user'->>'id_str' as retweeted_user_id_str,
tweet_body->'retweeted_status'->'user'->>'screen_name' as retweeted_user_screen_name,

tweet_body->'metadata'->'result_type' as result_type,
tweet_body->>'text' as text
FROM tweet
WHERE location_id = '{}'
AND tweet_body->>'language' = 'nl'
;
""".format(region)

In [8]:
import psycopg2
import pandas as pd

def tweets_to_pickle(region, name):
    # connect to DB
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD)
    cur = conn.cursor()
    
    circles = region.keys()
    print(circles)
    
    df_container = list()
    for r in circles:
        sql_query = region_sql_query(r)
        circle_tweets = pd.read_sql_query(sql_query, conn)
        circle_tweets.set_index('tweet_id')
        df_container.append(circle_tweets)

    tweets_database = pd.concat(df_container)
    print("Dataframe shape: {}".format(tweets_database.shape))
    print(tweets_database.info(memory_usage=True))
    
    cur.close()
    conn.close()
    
    tweets_database.to_pickle(path='tweets_database_{}.pickle'.format(name))

#tweets_to_pickle(region1, 'germany')
tweets_to_pickle(region2, 'netherlands')

dict_keys(['netherlands5', 'netherlands4', 'netherlands3', 'netherlands1', 'netherlands2'])
Dataframe shape: (0, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 18 columns):
tweet_id                      0 non-null object
inserted                      0 non-null object
location_query                0 non-null object
location_id                   0 non-null object
created_at                    0 non-null object
user_id_str                   0 non-null object
user_screen_name              0 non-null object
followers_count               0 non-null object
user_mention_screen_name      0 non-null object
user_mention_id_str           0 non-null object
in_reply_to_status_id         0 non-null object
in_reply_to_status_id_str     0 non-null object
in_reply_to_user_id_str       0 non-null object
in_reply_to_screen_name       0 non-null object
retweeted_user_id_str         0 non-null object
retweeted_user_screen_name    0 non-null object
result_type                